## ENDF export example

In this example we use `ncmat2endf` to export NCrystal data into the [ENDF-6 format](https://www.nndc.bnl.gov/endfdocs/ENDF-102-2023.pdf).

- https://github.com/mctools/ncrystal/wiki/ncmat2endf

- J.I. Marquez Damian, T. Kittelmann, D. Di Julio, S. Xu, D. Roubtsov and G. Muhrer. Open source pipelines for thermal scattering data generation: from NCrystal to ENDF-6, 16th Conference of Nuclear Data for Science and Technology, Madrid, June 2025.

- G. Schnabel, D. L. Aldama, R. Capote, https://doi.org/10.48550/arXiv.2312.08249


## Preamble

Install dependencies. Feel free to edit as you wish:

In [1]:
#By default we only do pip installs on Google Colab, but you
#can set the variable in the next line to True if you need it:
always_do_pip_installs = False
try:
    import google.colab as google_colab
except ModuleNotFoundError:
    google_colab=None#not on google colab
if always_do_pip_installs or google_colab:
    %pip -q install 'ncrystal[endf]'
#always import NCrystal:
import NCrystal as NC
assert NC.version_num >=  4001009, "too old NCrystal found"
NC.test() #< quick unit test that installation works

Tests completed succesfully


## Example: export ENDF-6 file from NCMAT

Ncmat2endf can convert most of the files in the NCrystal data library. Some of them cannot be converted either because they include physics that is not implemented in the ENDF-6 format, or require features that have not been yet implemented in ncmat2endf (e.g.: converting scattering kernels for gasses and liquids). The available data library can be listed with `NCrystal.browseFiles()` function, and it is available in the [NCrystal documentation](https://github.com/mctools/ncrystal/wiki/Data-library):

In [2]:
NC.browseFiles(dump=True, factory='stdlib')

==> 132 files from "stdlib" (stdlib, priority=120):
    AcrylicGlass_C5O2H8.ncmat
    AgBr_sg225_SilverBromide.ncmat
    Ag_sg225.ncmat
    Al2O3_sg167_Corundum.ncmat
    Al4C3_sg166_AluminiumCarbide.ncmat
    AlN_sg186_AluminumNitride.ncmat
    Al_sg225.ncmat
    Ar_Gas_STP.ncmat
    Au_sg225.ncmat
    BaF2_sg225_BariumFluoride.ncmat
    BaO_sg225_BariumOxide.ncmat
    Ba_sg229.ncmat
    Be3N2_sg206_BerylliumNitride.ncmat
    BeF2_sg152_Beryllium_Fluoride.ncmat
    BeO_sg186.ncmat
    Be_sg194.ncmat
    Bi_sg166.ncmat
    C_sg194_pyrolytic_graphite.ncmat
    C_sg227_Diamond.ncmat
    CaCO3_sg62_Aragonite.ncmat
    CaF2_sg225_CalciumFlouride.ncmat
    CaH2_sg62_CalciumHydride.ncmat
    CaO2H2_sg164_CalciumHydroxide.ncmat
    CaO_sg225_CalciumOxide.ncmat
    CaSiO3_sg2_Wollastonite.ncmat
    Ca_sg225.ncmat
    Ca_sg229_Calcium-gamma.ncmat
    CeO2_sg225_CeriumOxide.ncmat
    Cr_sg229.ncmat
    Cu2O_sg224_Cuprite.ncmat
    Cu_sg225.ncmat
    Dy2O3_sg206_DysprosiumOxide.ncmat
    Epoxy_Ar

To export ENDF files we need to import the `ncmat2endf` module of NCrystal and call the `ncmat2endf()` function. This function takes as required input a [configuration string](https://github.com/mctools/ncrystal/wiki/CfgRefDoc), which includes the ncmat file to convert as well as parameters that modify the material (temperature, density, which components of the material we want to output). As an example, this converts `LiH_sg225_LithiumHydride.ncmat`:

In [3]:
from NCrystal import ncmat2endf
res = ncmat2endf.ncmat2endf('LiH_sg225_LithiumHydride.ncmat;temp=293.6K',
                            material_name='LiH', force=True)

Initialise nuclear data...
Write ENDF file tsl_H_in_LiH.endf ...
Write ENDF file tsl_Li_in_LiH.endf ...
Files created:
  tsl_H_in_LiH.endf : H with fraction 1/2
  tsl_Li_in_LiH.endf : Li with fraction 1/2
Suggested material density: 0.7811126726 g/cm^3


The result of the execution is a dictionary with information about the material, which is useful to use these libraries in a radiation transport simulation:

In [4]:
import pprint
pprint.pprint(res)

{'density': 0.7811126726196552,
 'files': [{'component': 'H', 'file': 'tsl_H_in_LiH.endf', 'fraction': 0.5},
           {'component': 'Li', 'file': 'tsl_Li_in_LiH.endf', 'fraction': 0.5}],
 'temperature': 293.6}


In [5]:
def dump_endf_files(res, nlines=100):
  for entry in res['files']:
      print(80*'-')
      print(entry['file'])
      with open(entry['file']) as f:
          lines = [next(f) for _ in range(nlines)]
      print("".join(lines))
      print('--- file continues ---'.center(80))
      print(80*'-','\n\n')
dump_endf_files(res)

--------------------------------------------------------------------------------
tsl_H_in_LiH.endf
Created with ncmat2endf                                            999 0  0    0
 1.000000+3 9.993170-1         -1          0          0          0 999 1451    1
 0.000000+0 0.000000+0          0          0          0          6 999 1451    2
 1.000000+0 5.000000+0          0          0         12          1 999 1451    3
 0.000000+0 0.000000+0          0          0         65          3 999 1451    4
  1-H      MyLAB      EVAL-MMMYY NCrystal                          999 1451    5
 REFERENCE            DIST-MMMYY REV0-MMMYY                        999 1451    6
----MyLib             MATERIAL  999                                999 1451    7
-----THERMAL NEUTRON SCATTERING DATA                               999 1451    8
------ENDF-6 FORMAT                                                999 1451    9
****************************************************************** 999 1451   10
          

The file includes generic metadata ("MyLab", "MyLib"). To customize this the `ncmat2endf` module includes the `EndfMetaData` class:

In [6]:
m = ncmat2endf.EndfMetaData()
help(m)

Help on EndfMetaData in module NCrystal.ncmat2endf object:

class EndfMetaData(builtins.object)
 |  EndfMetaData(data=None)
 |
 |  Optional MetaData Parameters for the ENDF-6 file describing the origin
 |  and authorship of the file. For more information see the ENDF-6 format
 |  manual: https://www.nndc.bnl.gov/endfdocs/ENDF-102-2023.pdf
 |
 |  Methods defined here:
 |
 |  __init__(self, data=None)
 |      Initialise new EndfMetaData object, with default values of all
 |      parameters. If a data object is given in the form of a dictionary of
 |      (key,value) pairs, or another EndfMetaData object, the associated values
 |      will be updated accordingly.
 |
 |  __repr__(self)
 |      Return repr(self).
 |
 |  __str__(self)
 |      Return str(self).
 |
 |  get_value(self, name)
 |      Get value of parameter (naming is case insensitive).
 |
 |  set_all_dates_as_now(self)
 |      Set edate, ddate and rdate to the current date
 |
 |  set_value(self, name, value)
 |      Set value of

As an example, we will set the laboratory name to IAEA, the author to Marie Curie, and assign material number 3001 to Li in LiH, and 3002 to H in LiH. We will also set the evaluation, revision and distribution dates to the current month:

In [7]:
m.update_from_dict({'alab':'IAEA', 'auth': 'Marie Curie',
                    'matnum':{'Li':3001, 'H':3002},
                    'libname':'IAEA-TSL'})
m.set_all_dates_as_now()
res = ncmat2endf.ncmat2endf('LiH_sg225_LithiumHydride.ncmat;temp=293.6K',
                            endf_metadata=m,
                            material_name='LiH', force=True)
dump_endf_files(res, nlines=10)

Initialise nuclear data...
Write ENDF file tsl_H_in_LiH.endf ...
Write ENDF file tsl_Li_in_LiH.endf ...
Files created:
  tsl_H_in_LiH.endf : H with fraction 1/2
  tsl_Li_in_LiH.endf : Li with fraction 1/2
Suggested material density: 0.7811126726 g/cm^3
--------------------------------------------------------------------------------
tsl_H_in_LiH.endf
Created with ncmat2endf                                           3002 0  0    0
 1.000000+3 9.993170-1         -1          0          0          03002 1451    1
 0.000000+0 0.000000+0          0          0          0          63002 1451    2
 1.000000+0 5.000000+0          0          0         12          13002 1451    3
 0.000000+0 0.000000+0          0          0         65          33002 1451    4
  1-H      IAEA       EVAL-JUN25 Marie Curie                      3002 1451    5
 REFERENCE            DIST-JUN25 REV0-JUN25                       3002 1451    6
----IAEA-TSL          MATERIAL 3002                               3002 1451    7


## Command line interface

The same can be achieved using the command line interface. This is accessed with the `ncrystal_ncmat2endf` command:

In [8]:
!ncrystal_ncmat2endf -h

usage: ncrystal_ncmat2endf CFGSTR [--elas MODE] [--name NAME] [--mdata DATA]
                           [<<additional options described below>>]

Script for creating a set of ENDF-6 thermal scattering files for the material
described by a particular NCrystal cfg-string.

The script uses the endf-parserpy package from IAEA to format and check the syntax
of the ENDF-6 file:

G. Schnabel, D. L. Aldama, R. Capote, https://doi.org/10.48550/arXiv.2312.08249

Note that while the handling of multiple temperatures in one ENDF-6 file is
supported via the --othertemps keyword, it is not recommended. This is because
NCrystal computes an optimal (alpha, beta) grid for each material and temperature,
while the ENDF format imposes the same grid on all temperatures.

Example invocations:

    $> ncrystal_ncmat2endf  "Al_sg225.ncmat;temp=350K"

    $> ncrystal_ncmat2endf  "Si_sg227.ncmat;temp=293.6K" -m MATNUM:Si:99 --now

    $> ncrystal_ncmat2endf  "ZnO_sg186_ZincOxide.ncmat;temp=293.15K" -n ZnO -e \


ENDF metadata can be provided in the command line using a JSON dictionary or adding individual items:

In [9]:
!ncrystal_ncmat2endf --mdata help

Meta-data for ENDF can be provided by the --mdata option, by specifying a JSON
dictionary like:

  --mdata='{ "LIBNAME" : "MySuperLib", "ALAB" : "MySuperLab" }'

Or by adding individual items with the -m option like:

  -m LIBNAME:MySuperLib -m AUTH:"J. Chadwick"

The list of supported meta-data keys and their meaning is:

        ALAB : Mnemonic for the originating laboratory.
        AUTH : Author(s) name(s).
     LIBNAME : Name of the nuclear data library.
        NLIB : Nuclear data library identifier (e.g. NLIB=0 for ENDF/B).
   REFERENCE : Primary reference for the evaluation.
        LREL : Nuclear data library release number.
        NVER : Nuclear data library version number.
      MATNUM : ENDF-6 MAT number assignment. Dictionary with string keys and int
               values that maps element symbols to MAT numbers, such as
               {"Zn":101,"O":102}. When setting this parameter it is also
               allowed to use a single string formatted like "Zn:101,O:102" See

In [10]:
!ncrystal_ncmat2endf 'LiH_sg225_LithiumHydride.ncmat;temp=293.6K' --name 'LiH' -f --now \
      -m alab:IAEA -m auth:"Marie Curie" -m matnum:"Li:3001,H:3002" -m libname:'IAEA-TSL' \
      --index results.json

Initialise nuclear data...
Write ENDF file tsl_H_in_LiH.endf ...
Write ENDF file tsl_Li_in_LiH.endf ...
Files created:
  tsl_H_in_LiH.endf : H with fraction 1/2
  tsl_Li_in_LiH.endf : Li with fraction 1/2
Suggested material density: 0.7811126726 g/cm^3
Writing index file: results.json


The composition data that is returned by the `ncmat2endf()` function can be output in the command line using the `--index` option:

In [11]:
!cat results.json

{
    "density": 0.7811126726196552,
    "temperature": 293.6,
    "files": [
        {
            "file": "tsl_H_in_LiH.endf",
            "fraction": 0.5,
            "component": "H"
        },
        {
            "file": "tsl_Li_in_LiH.endf",
            "fraction": 0.5,
            "component": "Li"
        }
    ]
}
